In [29]:
!pip install emoji

In [30]:
# Import các thư viện cần thiết
import re
import pandas as pd
import emoji

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
# Đọc danh sách teencode từ file teencode.txt
teencode_dict = {}
with open(r'/content/drive/MyDrive/NLP/teencode.txt', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        parts = line.strip().split(': ')
        if len(parts) == 2:
            teencode_dict[parts[0]] = parts[1]

# Đọc dữ liệu từ file CSV
data = pd.read_csv(r"/content/drive/MyDrive/NLP/labeled_sentiment_data.csv")

# Hiển thị dữ liệu ban đầu
print("Dữ liệu ban đầu:")
data.head()

Dữ liệu ban đầu:


,comment,hospital,star,rating,sentiment
0,"Mình đã có 2 bé, muốn sinh thêm bé nữa nhưng v...",Bệnh Viện Đa khoa Hà Nội - Hanoi Hospital,5 sao,"3,6",Tốt
1,hôm 04/11 toi đưa vợ đi khám tiền hôn nhan ivf...,Bệnh Viện Đa khoa Hà Nội - Hanoi Hospital,5 sao,"3,6",Tốt
2,Bệnh viện tuy nhỏ nhưng sạch đẹp. Nữ bác sĩ kh...,Bệnh Viện Đa khoa Hà Nội - Hanoi Hospital,5 sao,"3,6",Tốt
3,Tháng 9 2vck mình có đi thăm khám SKSS và có n...,Bệnh Viện Đa khoa Hà Nội - Hanoi Hospital,2 sao,"3,6",Tệ
4,"Mình đã khám ở viện rồi ạ , nhân viên tư vấn v...",Bệnh Viện Đa khoa Hà Nội - Hanoi Hospital,5 sao,"3,6",Tốt


In [33]:
teencode_dict

{':v': 'hihi',
 '<3': 'yêu',
 '♥️': 'yêu',
 '❤': 'yêu',
 'a': 'anh',
 'ac': 'anh chị',
 'ace': 'anh chị em',
 'ad': 'quản lý',
 'ae': 'anh em',
 'ah': 'à',
 'ak': 'à',
 'amin': 'quản lý',
 'androir': 'android',
 'app': 'ứng dụng',
 'auto ': 'tự động',
 'ây': 'vậy',
 'b nào': 'bạn nào',
 'bằg': 'bằng',
 'băng': 'bằng',
 'băp': 'bắp',
 'băt': 'bắt',
 'battery': 'pin',
 'bể': 'vỡ',
 'been': 'bên',
 'best': 'tốt nhất',
 'bgqafy\xa0': 'ngày',
 'bh': 'bảo hiểm',
 'bhx': 'bảo hành',
 'bi': 'bị',
 'big': 'lớn',
 'bik': 'biết',
 'bin': 'pin',
 'bit': 'biết',
 'bít': 'biết',
 'bn': 'bệnh nhân',
 'bông tróc': 'bong tróc',
 'k': 'không',
 'ok': 'được',
 'bro': 'anh em',
 'bt': 'biết',
 'bth': 'bình thường',
 'bthg': 'bình thường',
 'bua': 'bữa',
 'bùn': 'buồn',
 'buonc': 'buồn',
 'bx': 'bệnh xạ',
 'bye': 'tạm biệt',
 'c': 'chị',
 'cac': 'các',
 'cam': 'máy ảnh',
 'card': 'thẻ',
 'châu': 'khỏe',
 'chiệu': 'triệu',
 'chíp': 'chip',
 'chội': 'trội',
 'chs': 'chơi',
 'chửa': 'chữa',
 'chug\xa0': 'chun

In [34]:
# Hàm trích xuất emoji từ văn bản
def extract_emojis(text):
    return ''.join(c for c in text if c in emoji.EMOJI_DATA)

# Hàm thay thế teencode bằng từ chuẩn (đảm bảo thay thế toàn bộ từ teencode)
def replace_teencode(text):
    # Chuyển văn bản thành chữ thường để việc thay thế chính xác
    text = text.lower()

    # Thay thế các từ teencode bằng từ chuẩn
    for teencode, replacement in teencode_dict.items():
        if re.match(r'^\w+$', teencode):  # Nếu là "từ" bình thường (chỉ gồm chữ cái, số)
            text = re.sub(r'\b' + re.escape(teencode) + r'\b', replacement, text)
        else:  # Nếu là ký hiệu đặc biệt như <3, :v,...
            text = text.replace(teencode, replacement)
    return text

# Hàm làm sạch văn bản
def clean_text(text):
    # Xóa emoji khỏi văn bản gốc
    text = emoji.replace_emoji(text, replace='')

    # Thay thế teencode bằng từ đầy đủ
    text = replace_teencode(text)

    # Xóa dấu chấm lửng, ký tự lạ
    text = re.sub(r"…", "", text)  # Thay thế dấu chấm lửng
    text = re.sub(r"[^\w\s,.?!\-]", "", text)  # Xóa ký tự lạ, giữ lại chữ cái, số, khoảng trắng và dấu cơ bản

    # Loại bỏ URL
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)

    # Chuẩn hóa dấu câu (bỏ dấu thừa, chuẩn hóa khoảng trắng)
    text = re.sub(r"\s+", " ", text)  # Thay thế nhiều khoảng trắng bằng 1
    text = text.strip()  # Xóa khoảng trắng đầu/cuối

    # Chuyển về chữ thường
    text = text.lower()

    return text

In [35]:
# Nhập một câu chứa teencode để kiểm tra
test_text = "Sớm mai, đi Cafe zìa <3, chạy ngang trên đường 3/2, quận Ninh Kiều, thấy Bệnh viện Đa khoa Quốc tế Vinmec Cần Thơ đang trong thời gian hoàn thiện.\nVinmec Cần Thơ có không gian khám chữa bệnh hiện đại, tiện nghi và thoải …"

# Kiểm tra việc thay thế teencode
replaced_text = replace_teencode(test_text)

# In kết quả trước và sau khi thay thế teencode
print("Câu gốc:", test_text)
print("Câu sau khi thay thế teencode:", replaced_text)


Câu gốc: Sớm mai, đi Cafe zìa <3, chạy ngang trên đường 3/2, quận Ninh Kiều, thấy Bệnh viện Đa khoa Quốc tế Vinmec Cần Thơ đang trong thời gian hoàn thiện.
Vinmec Cần Thơ có không gian khám chữa bệnh hiện đại, tiện nghi và thoải …
Câu sau khi thay thế teencode: sớm mai, đi cafe về yêu, chạy ngang trên đường 3/2, quận ninh kiều, thấy bệnh viện đa khoa quốc tế vinmec cần thơ đang truyền thời gian hoàn thiện.
vinmec cần thơ có không gian khám chữa bệnh hiện đại, tiện nghi và thoải …


In [36]:
# Áp dụng hàm thay thế teencode cho mỗi câu trong cột 'comment'
data['cleaned_comment'] = data['comment'].apply(lambda x: replace_teencode(x))

# In ra dữ liệu trước và sau khi loại bỏ teencode
print("Dữ liệu trước và sau khi loại bỏ teencode:")
data[['comment', 'cleaned_comment']].tail()  # Hiển thị 5 dòng đầu tiên để kiểm tra


Dữ liệu trước và sau khi loại bỏ teencode:


,comment,cleaned_comment
3058,"Bệnh viện rộng rãi, không đông người. Bảo vệ, ...","bệnh viện rộng rãi, không đông người. bảo vệ, ..."
3059,"Sớm mai, đi Cafe zìa, chạy ngang trên đường 3/...","sớm mai, đi cafe về, chạy ngang trên đường 3/2..."
3060,Có ai biết khi nào bệnh viện mở lại không mình...,có ai biết khi nào bệnh viện mở lại không mình...
3061,Bệnh viện Quốc tế tại Cần Thơ.,bệnh viện quốc tế tại cần thơ.
3062,"Bệnh viện tư, gần cầu Quang Trung","bệnh viện tư, gần cầu quang trung"


In [37]:

# Lưu dữ liệu đã làm sạch
data.to_csv("/content/drive/MyDrive/NLP/processed_sentiment_data_no_emojis.csv", index=False)

# In ra thông báo hoàn thành
print("\nDữ liệu đã được lưu vào file processed_sentiment_data_no_emojis.csv")



Dữ liệu đã được lưu vào file processed_sentiment_data_no_emojis.csv
